In [1]:
%% morning matlab %%

## Convert from BCI2000

In [2]:
%% BCI2000 to matlab conversion

addpath(genpath('/home/brooke/knight_lab_commons/loading_data_scripts/BCI2000mex'))
path_data = '/home/brooke/knight_server/remote/WashU/data/PacmanTask/BJH016/PacmanTask/ECOGS001R01.dat'
[signal,states,parameters] = load_bcidat(path_data);


path_data =

    '/home/brooke/knight_server/remote/WashU/data/PacmanTask/BJH016/PacmanTask/ECOGS001R01.dat'



In [3]:
%% Now add other paths %%
% add fieldtrip
ftDir = '/home/brooke/knight_server/remote/lbellier/DataWorkspace/_tools/git/fieldtrip/';
if exist('ft_defaults.m', 'file') == 0
    addpath(ftDir); ft_defaults;
end
% Ludo's tools
addpath('/home/brooke/knight_server/remote/lbellier/DataWorkspace/_tools/');

%% Add 'photodiode'
signal_photo = [signal, states.Trial_on_off];
parameters.ChannelNames.Value{273} =  'photodiode';

%% Convert to fieldtrip
data = ft_array2data(double(signal_photo),  parameters.SamplingRate.NumericValue, parameters.ChannelNames.Value);

%% save %%
save('/home/brooke/knight_server/remote/bstavel/pacman_ieeg_data/BJH016/BJH016_raw_data.mat', 'data', '-v7.3');

-------------------------------------------------------------------------------------------
FieldTrip is developed by members and collaborators of the Donders Institute for Brain,
Cognition and Behaviour at Radboud University, Nijmegen, the Netherlands.

                          --------------------------
                        /                            \
                     ------------------------------------
                    /                                    \
          -------------------------------------------------
         /                            /\/\/\/\/\ 
         ---------------------------------------------------
                  |        F  i  e  l  d  T  r  i  p       |
                  ------------------------------------------
                   \                                      /
                     ------------------------------------
                          \            /
                            ----------

Please cite the FieldTrip re

## After conversion from BCI2000

begin here after data from BCI2000 has been converted to fieldtrip

In [8]:
restoredefaultpath

In [9]:
%% add field trip %%
ftDir = '/home/brooke/fieldtrip/fieldtrip-20191213';
if exist('ft_defaults.m', 'file') == 0
    addpath(ftDir); ft_defaults;
end


In [6]:
%% load data %%

% paths %
base_dir = '/home/brooke/knight_server/remote/bstavel/'
bj16_path = '/home/brooke/knight_server/remote/bstavel/pacman_ieeg_data/BJH016/BJH016_raw_data.mat'

%% loda data %%
load(bj16_path)


base_dir =

    '/home/brooke/knight_server/remote/bstavel/'


bj16_path =

    '/home/brooke/knight_server/remote/bstavel/pacman_ieeg_data/BJH016/BJH016_raw_data.mat'



In [15]:
data


data = 

  struct with fields:

       fsample: 2000
    sampleinfo: [1 5901900]
         trial: {[273x5901900 uint8]}
          time: {[0 5.0000e-04 1.0000e-03 0.0015 0.0020 0.0025 0.0030 ... ]}
         label: {273x1 cell}



In [10]:
%% Downsample %%
cfg = [];
cfg.resamplefs = 1080;
data = ft_resampledata(cfg, data);

the input is raw data with 273 channels and 1 trials
the call to "ft_selectdata" took 0 seconds and required the additional allocation of an estimated 1 MB
resampling data
resampling data in trial 1 from 1
original sampling rate = 2000 Hz
new sampling rate = 1080 Hz
the call to "ft_resampledata" took 28 seconds and required the additional allocation of an estimated 18628 MB


In [11]:
%% Detrend and notch clinical data
HPcutoff = 1; % high-pass filter cutoff
LPcutoff = 200; % low-pass filter cutoff change to 150
powerLineF0 = 60;
cfg = [];
cfg.bsfilter = 'yes';
cfg.bsfreq = (powerLineF0 * (1:round((LPcutoff+powerLineF0)/powerLineF0)) + [-1; 1])';
if data.fsample <= 512
    cfg.bsfreq = cfg.bsfreq(1:end-1, :); % for fs=512Hz, e.g. OS13, OS21
end
cfg.bsfiltord = 3;
cfg.hpfilter = 'yes';
cfg.hpfreq = HPcutoff;
cfg.hpfiltord = 3;
cfg.lpfilter = 'yes';
cfg.lpfreq = LPcutoff;
cfg.lpfiltord = 3;
filtered_data = ft_preprocessing(cfg, data);


 In '/home/brooke/fieldtrip/fieldtrip-20191213/utilities/private/fixsampleinfo.m' at line 80
 In '/home/brooke/fieldtrip/fieldtrip-20191213/utilities/ft_datatype_raw.m' at line 147
 In '/home/brooke/fieldtrip/fieldtrip-20191213/utilities/ft_checkdata.m' at line 268
 In '/home/brooke/fieldtrip/fieldtrip-20191213/ft_preprocessing.m' at line 281

> In fixsampleinfo (line 94)
In ft_datatype_raw (line 147)
In ft_checkdata (line 268)
In ft_preprocessing (line 281)
the call to "ft_selectdata" took 0 seconds and required the additional allocation of an estimated 2 MB
preprocessing
preprocessing trial 1 from 1
the call to "ft_preprocessing" took 146 seconds and required the additional allocation of an estimated 6705 MB


In [12]:
%plot native
%% display %%
cfg = [];
cfg.viewmode = 'vertical';
cfg.preproc.demean = 'yes';
% cfg.blocksize = 10;
% cfg.channel = goodChansDispIdx;
% cfg.position = pos2;
% cfg.ylim = [-1 1] * .0005;
ft_databrowser(cfg, filtered_data);

the input is raw data with 273 channels and 1 trials
detected   0 visual artifacts
the call to "ft_prepare_layout" took 0 seconds and required the additional allocation of an estimated 1 MB
the call to "ft_databrowser" took 8 seconds and required the additional allocation of an estimated 6772 MB


In [ ]:
%% display %%
cfg = [];
cfg.viewmode = 'vertical';
cfg.preproc.demean = 'yes';
% cfg.blocksize = 10;
% cfg.channel = goodChansDispIdx;
% cfg.position = pos2;
% cfg.ylim = [-1 1] * .0005;
ft_databrowser(cfg, data);

In [ ]:
data

# Cleaning Notes with Bob
## 1/19/2022

Clean data, but with a lot of spiking activity

Epileptic channels: Gl1, GL2, GL3, HL1, HL2

Noisy channels: BL1, F8, EL1, EL2 , HL14, HL15, HL16

Epochs that may need to be removed (depending on how the data looks after removing epileptic channels + nearby channels): LOOK AT THIS AGAIN FOR THIS PATEINT
`116-117, 127.5:129, 160-161.5, 343-345, 540-541, 6216-623, 681-683`
